In [ ]:
install.packages("reticulate")
install.packages("jsonlite")
install.packages("data.table")
install.packages("quanteda")
install.packages("Hmisc")
install.packages("lmSupport")


In [ ]:
# ---------------------------------------------

In [3]:
# 加载所需的R包
library(reticulate)
library(jsonlite)
library(data.table)
library(quanteda)

# 使用 Conda 环境
use_condaenv("agent", required = TRUE)

# 检查是否正确设置了 Python 环境
py_config()

# 读取文件夹中的所有txt文件
file_list <- list.files(path = "F:/Paper_goal/MKT_research/RA/PKU-park/dropbox/LIWC/Study 3/data", pattern = "*.txt", full.names = TRUE)
study3 <- data.table()

for (file in file_list) {
  text <- readLines(file, warn = FALSE)
  text <- paste(text, collapse = " ")
  study3 <- rbind(study3, data.table(Tweet = text, FileName = basename(file)))
}

# 读取并清理Brysbaert具体性分数
brysbaert <- read.table(file = 'F:/Paper_goal/MKT_research/RA/PKU-park/dropbox/LIWC/R_Functions/brysbaert.txt', sep = '\t', header = TRUE)
brysbaert$Conc.M <- as.numeric(brysbaert$Conc.M)  # 确保Conc.M列是数值类型
brysbaert <- brysbaert[!is.na(brysbaert$Conc.M), ]

# 计算BCI值的函数
brysbaert.calculator <- function(text, keep){
  text <- as.character(text)
  brys.output <- as.data.frame(matrix(NA,
                                      ncol = 3,
                                      nrow = length(text)))
  colnames(brys.output) <- c("newtext", "wc", "bryscore")
  brys.output$bryscore <- 0
  for (i in 1:length(text)){
    if (i %% 100 == 0){
      print(paste0(i," done"))
    }
    words <- strsplit(text[i], " ")[[1]]
    words <- sapply(words, tolower)
    brys.output$newtext[i] <- paste(words[words %in% keep],
                                    collapse = " ")
    words <- strsplit(brys.output$newtext[i], " ")[[1]]
    for (w in words){
      index <- which(!is.na(match(brysbaert$Word, w)))
      weight <- brysbaert$Conc.M[index]
      brys.output$bryscore[i] <- brys.output$bryscore[i] + weight
    }
  }
  bryscorpus <- corpus(brys.output$newtext)
  brystokens <- tokens(bryscorpus)
  brysdfm <- dfm(brystokens)
  brys.output$wc <- rowSums(brysdfm)
  brys.output$bryscore <- brys.output$bryscore / brys.output$wc
  return(brys.output$bryscore)
}

# 应用到study3数据集
study3$bryscore <- brysbaert.calculator(study3$Tweet, brysbaert$Word)
study3$bryscorer <- 6 - study3$bryscore
study3$zbryscorer <- scale(study3$bryscorer, center = TRUE, scale = TRUE)

# 汇总结果到JSON文件
results <- list(
  bryscorer = study3$bryscorer
)

write_json(results, path = "F:/Paper_goal/MKT_research/RA/PKU-park/dropbox/LIWC/Study 3/results.json")


python:         D:/Anaconda/anaconda_software/envs/agent/python.exe
libpython:      D:/Anaconda/anaconda_software/envs/agent/python39.dll
pythonhome:     D:/Anaconda/anaconda_software/envs/agent
version:        3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]
Architecture:   64bit
numpy:          D:/Anaconda/anaconda_software/envs/agent/Lib/site-packages/numpy
numpy_version:  1.23.4

NOTE: Python version was forced by use_python() function